In [1]:
import pickle
import re
import numpy as np
import ast

In [2]:
from os import listdir
import glob


In [17]:
file_names = glob.glob('../../iclr_result_dirs/teacher_student/out*')

In [18]:
len(file_names)

555

In [19]:
file_name='../../iclr_result_dirs/teacher_student/out_traject.831800'

In [20]:
def vanilla_out_parser(file_name):    
    count_test_results = 0
    done_config = False
    accu_list=[]
    with open(file_name,'r') as f:
        for line in f:
            if not done_config and line[:6]=='config':
                config_dict_str = line[7:].lstrip()
                done_config = True
            if line[:6] == 'python':
                python_commant = line
            if count_test_results > 0:
                accu = line.split(':')[-1]
                accu_list.append(float(accu))
                count_test_results -= 1
            if 'running 5 times on test data' in line:
                count_test_results = 5

    # print(accu_list)
    if len(accu_list)>0:
        config_dict = ast.literal_eval(config_dict_str)
#         print(config_dict)
#         print(np.mean(accu_list))
        config_dict['accuracy'] = np.mean(accu_list)
        config_dict['accuracy_std'] = np.std(accu_list)
#         config_dict['accuracy'] = np.mean(accu_list)
        return config_dict
    else:
        return None


In [21]:
records = []
for ii,file_name in enumerate(file_names):
    conf=vanilla_out_parser(file_name)
    if conf is not None:
        conf['file_name']=file_name.split('/')[-1]
        conf['file_name_full']=file_name
        conf['teacher_short']=conf['teacher_net'][-25:]
#         print(conf['teacher_short'])
        records.append(conf)

In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame(records)

In [24]:
df.columns

Index(['accuracy', 'accuracy_std', 'batch_norm_student', 'broadcast',
       'conv_rnn_type', 'data_augmentation', 'dataset_center', 'dataset_norm',
       'decoder_epochs', 'decoder_optimizer', 'dense_interface', 'dropout',
       'dropout1', 'dropout2', 'epochs', 'file_name', 'file_name_full',
       'fine_tune_student', 'height_shift_range', 'int_epochs',
       'last_layer_size', 'last_maxpool_en', 'layer_norm_2', 'layer_norm_res',
       'layer_norm_student', 'learning_patience', 'loss', 'manual_suffix',
       'max_length', 'n_classes', 'n_samples', 'nl', 'noise', 'num_feature',
       'pos_det', 'pretrained_student_path', 'random_n_samples', 'res',
       'resblocks', 'resnet_mode', 'rnn_dropout', 'rnn_layer1', 'rnn_layer2',
       'rotation_range', 'run_index', 'run_name_prefix', 'shuffle_traj',
       'skip_conn', 'skip_student_training', 'snellen', 'stopping_patience',
       'student_block_size', 'student_nl', 'student_version', 'style',
       'syclopic_norm', 'teacher_net'

In [25]:
df.sort_values(by=['vm_kappa']).loc[df['style']=='xx1_vonmises_walk'].loc[:,['file_name','style','vm_kappa', 'accuracy']]

,file_name,style,vm_kappa,accuracy
25,out_traject.775281,xx1_vonmises_walk,-5.0,0.70802
39,out_traject.961905,xx1_vonmises_walk,-5.0,0.70564
1,out_traject.775336,xx1_vonmises_walk,-2.0,0.71188
49,out_traject.744841,xx1_vonmises_walk,-2.0,0.70392
93,out_traject.775389,xx1_vonmises_walk,-1.0,0.69300
99,out_traject.775394,xx1_vonmises_walk,-1.0,0.70392
85,out_traject.743838,xx1_vonmises_walk,0.0,0.68472
126,out_traject.961504,xx1_vonmises_walk,0.0,0.67748
84,out_traject.742837,xx1_vonmises_walk,2.0,0.65910
11,out_traject.775471,xx1_vonmises_walk,2.0,0.65798


In [26]:
df.loc[df['style']=='xx1_vonmises_walk']

,accuracy,accuracy_std,batch_norm_student,broadcast,conv_rnn_type,data_augmentation,dataset_center,dataset_norm,decoder_epochs,decoder_optimizer,...,time_pool,time_sec,traj_out_scale,trajectories_num,trajectory_index,upsample,val_set_mult,verbose,vm_kappa,width_shift_range
1,0.71188,0.001525,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,-2.0,0.1
3,0.65800,0.003481,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,5.0,0.1
11,0.65798,0.002218,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,2.0,0.1
14,0.65522,0.002846,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,5.0,0.1
25,0.70802,0.002658,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,-5.0,0.1
39,0.70564,0.001297,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,-5.0,0.1
49,0.70392,0.002485,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,-2.0,0.1
71,0.65950,0.002125,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,20.0,0.1
84,0.65910,0.002616,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,2.0,0.1
85,0.68472,0.001883,False,1,gru,True,True,128.0,40,SGD,...,average_pool,0.3,4.0,-1,0,7,1.0,2,0.0,0.1


In [293]:
df.sort_values(by=['n_classes']).loc[df['style']=='spiral_2dir2'].loc[:,['file_name','teacher_short','style','resnet_mode',
                                                                         'student_block_size',
                                                                         'broadcast','n_samples','n_classes','epochs', 'accuracy']]

,file_name,teacher_short,style,resnet_mode,student_block_size,broadcast,n_samples,n_classes,epochs,accuracy
46,out_traject.7526,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.92444
32,out_traject.956103,del_52191__1631198121.hdf,spiral_2dir2,False,2,1,5,10.0,100,0.84594
35,out_traject.956123,del_52191__1631198121.hdf,spiral_2dir2,False,2,1,10,10.0,100,0.86538
38,out_traject.59691,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,2,1,5,10.0,100,0.93388
43,out_traject.831470,el_846962__1632251071.hdf,spiral_2dir2,True,1,1,10,10.0,100,0.94866
28,out_traject.5936,snet/ref_cifar100_02.hdf/,spiral_2dir2,True,1,0,5,10.0,100,0.86200
48,out_traject.7065,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,0,5,10.0,100,0.87450
72,out_traject.56703,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,1,10,10.0,100,0.94654
53,out_traject.831336,el_846962__1632251071.hdf,spiral_2dir2,True,1,1,5,10.0,100,0.92068
66,out_traject.7527,esnet/ref_cifar10_03.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.92266


In [294]:
df.sort_values(by=['n_classes']).loc[df['style']=='spiral_2dir2'].loc[df['resnet_mode']].loc[:,['file_name','teacher_short','style','resnet_mode',
                                                                         'student_block_size',
                                                                         'broadcast','n_samples','n_classes','epochs', 'accuracy']]

,file_name,teacher_short,style,resnet_mode,student_block_size,broadcast,n_samples,n_classes,epochs,accuracy
46,out_traject.7526,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.92444
38,out_traject.59691,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,2,1,5,10.0,100,0.93388
43,out_traject.831470,el_846962__1632251071.hdf,spiral_2dir2,True,1,1,10,10.0,100,0.94866
28,out_traject.5936,snet/ref_cifar100_02.hdf/,spiral_2dir2,True,1,0,5,10.0,100,0.86200
48,out_traject.7065,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,0,5,10.0,100,0.87450
72,out_traject.56703,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,1,10,10.0,100,0.94654
53,out_traject.831336,el_846962__1632251071.hdf,spiral_2dir2,True,1,1,5,10.0,100,0.92068
66,out_traject.7527,esnet/ref_cifar10_03.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.92266
25,out_traject.225256,esnet/ref_cifar10_03.hdf/,spiral_2dir2,True,1,0,10,10.0,100,0.89680
10,out_traject.831388,el_846962__1632251071.hdf,spiral_2dir2,True,2,1,5,10.0,100,0.93342


In [280]:
df.sort_values(by=['n_classes']).loc[df['style']=='spiral_2dir2'].loc[df['resnet_mode']==False].loc[:,['file_name','teacher_short','style','resnet_mode',
                                                                         'student_block_size',
                                                                         'broadcast','n_samples','n_classes','epochs', 'accuracy']]

,file_name,teacher_short,style,resnet_mode,student_block_size,broadcast,n_samples,n_classes,epochs,accuracy
29,out_traject.956103,del_52191__1631198121.hdf,spiral_2dir2,False,2,1,5,10.0,100,0.84594
32,out_traject.956123,del_52191__1631198121.hdf,spiral_2dir2,False,2,1,10,10.0,100,0.86538
42,out_traject.956166,del_52191__1631198121.hdf,spiral_2dir2,False,2,0,10,10.0,100,0.84638
94,out_traject.956187,del_52191__1631198121.hdf,spiral_2dir2,False,1,0,10,10.0,100,0.83422


In [281]:
df.to_pickle('DRC_results_Oct3_21.pkl')

In [282]:
700*100/3600

19.444444444444443

In [27]:
df.sort_values(by=['n_classes']).loc[df['style']=='xx1_not_moving'].loc[:,['file_name','teacher_short','style','resnet_mode',
                                                                         'student_block_size',
                                                                         'broadcast','n_samples','n_classes','epochs', 'accuracy']]

,file_name,teacher_short,style,resnet_mode,student_block_size,broadcast,n_samples,n_classes,epochs,accuracy
46,out2_net.567854,esnet/ref_cifar10_02.hdf/,xx1_not_moving,True,1,1,5,10.0,100,0.7612
63,out2_net.567860,esnet/ref_cifar10_03.hdf/,xx1_not_moving,True,1,1,5,10.0,100,0.7603
69,out2_net.567847,esnet/ref_cifar10_01.hdf/,xx1_not_moving,True,1,1,5,10.0,100,0.7546
94,out2_net.567839,snet/ref_cifar100_03.hdf/,xx1_not_moving,True,1,1,5,100.0,100,0.5207
7,out2_net.567835,snet/ref_cifar100_02.hdf/,xx1_not_moving,True,1,1,5,100.0,100,0.5207
8,out2_net.567829,snet/ref_cifar100_01.hdf/,xx1_not_moving,True,1,1,5,100.0,100,0.5193


In [29]:
df.sort_values(by=['n_classes']).loc[df['student_version']==103].loc[df['epochs']==100].loc[:,['file_name','teacher_short','style','resnet_mode',
                                                                         'student_block_size',
                                                                         'broadcast','n_samples','n_classes','epochs', 'accuracy']]

,file_name,teacher_short,style,resnet_mode,student_block_size,broadcast,n_samples,n_classes,epochs,accuracy
42,out2_net.567117,esnet/ref_cifar10_01.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.77764
44,out2_net.567125,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.78104
35,out2_net.299627,esnet/ref_cifar10_02.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.78006
33,out2_net.567136,esnet/ref_cifar10_03.hdf/,spiral_2dir2,True,1,1,5,10.0,100,0.78024
90,out2_net.567155,snet/ref_cifar100_02.hdf/,spiral_2dir2,True,1,1,5,100.0,100,0.53840
103,out2_net.567162,snet/ref_cifar100_03.hdf/,spiral_2dir2,True,1,1,5,100.0,100,0.54038


In [237]:
conf['file_name_full']

TypeError: 'NoneType' object is not subscriptable